# Exploratory analysis for machine learning algorithm -Random forest- for coral bleaching classification 

## Load dataset

In [ ]:
'''
    Import libraries
'''
from sklearn import datasets
import pandas as pd
import numpy as np
import sklearn
import seaborn as sb
import matplotlib.pyplot as plt
from numpy import mean 
from numpy import std
import pingouin as pg
from scipy.stats import shapiro
from scipy.stats import levene 
from scipy.stats import bartlett
from scipy.stats import kruskal
from scipy.stats import mannwhitneyu
import scikit_posthocs as sp 
from scipy import stats
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.ensemble import RandomForestClassifier
# check scikit-learn version
print(sklearn.__version__)

In [ ]:
'''
    Load full dataset
'''
data = pd.read_csv('df_sst_clouds.csv')
#data.head(5)  

In [ ]:
'''
    Subset DF by SEVERITY_CODE [0,1,2,3]
'''
#data = data.dropna() # drop rows that contains NaN's 
data = data[(data.SEVERITY_CODE == 0)|(data.SEVERITY_CODE == 1)|(data.SEVERITY_CODE == 2)|(data.SEVERITY_CODE == 3)] 
#data = data[(data.YEAR >= 2005)] # First year with more than 100 records
#list(data.columns)
data = data.dropna()
len(data)

In [ ]:
'''
    ADD colums grouped by SEVERITY CODE combinations
        Here the lenght of the DF is kept
'''
data['SEVERITY_CODE2'] = np.where(data['SEVERITY_CODE']==0, "0","123") 
data['SEVERITY_CODE3'] = np.where(data['SEVERITY_CODE']<=1,"01","23")
data['SEVERITY_CODE5'] = np.where(data['SEVERITY_CODE']<=2,"012","3")

In [ ]:
# # data = data[data.duplicated(['lat','lon','full_date',
# # 'CF','SST','JD','SEASON','DHW','DAMMM'])]
#data = data.drop_duplicates(['lat','lon','JD','full_date','CF','SST','DHW','WD','CD'])
#data = data.drop(data[(data.DHW == 0) & (data.DHWrunmean7 == 0) & (data.DHWrunmean30 == 0) &(data.DHWrunmean90 == 0) & (data.SEVERITY_CODE > 0)].index) # restricting to cells with an actual DHW value only
len(data)

In [ ]:
'''
    Define dataset(s) for correllogram DAY of REPORT "dor"
'''
data_dor = data.loc[:, ~data.columns.str.contains('adj')] # select only day of freport
data_dor = data_dor.loc[:, ~data_dor.columns.str.contains('_a')] # drop "CF_a..." variables
X = data_dor.drop(['DHW_class','SEVERITY_CODE','SEVERITY_CODE2','SEVERITY_CODE3', 'COUNTRY', 'DAY', 'MONTH','YEAR','JD','lat','lon','full_date','ITEM_ID','SST_std7','SST_std30','SST_std90'], axis=1) # dependent variables
y = data_dor['SEVERITY_CODE'] # labels (indipendent variable)

In [ ]:
'''
    Define dataset(s) for correllogram DAY of REPORT "anomalies"
'''
data_dor = data.loc[:, ~data.columns.str.contains('adj')] # select only day of freport
#data_dor = data_dor.loc[:, ~data_dor.columns.str.contains('_a')] # drop "CF_a..." variables
X = data_dor.drop(['DHW_class','SEVERITY_CODE2','SEVERITY_CODE3', 'COUNTRY', 'DAY', 'MONTH','YEAR','JD','lat','lon','full_date','ITEM_ID','SST_std7','SST_std30'], axis=1) # dependent variables
y = data_dor['SEVERITY_CODE'] # labels (indipendent variable)

In [ ]:
'''
    Correlation plot for date of report
'''
pearsoncorr = X.corr(method='pearson')
#spaermancorr
plt.figure(figsize=(20,15))
sb.heatmap(pearsoncorr, 
            vmin=-1, vmax=1, center=0,
            xticklabels=pearsoncorr.columns,
            yticklabels=pearsoncorr.columns,
            cmap='coolwarm',
            #annot=True,
            linewidth=0.5,
            fmt='.2g')
plt.savefig('corr_plot.pdf', dpi=300)
#plt.show()

In [ ]:
'''
Pearson significance
'''
from scipy.stats.stats import pearsonr
pval = X.rcorr(stars=False)
#pval.to_csv('corr_pval_features.csv')
pval

In [ ]:
'''
Histogram (all together)
''' 
X.hist(figsize=(20,17))
plt.show()
#plt.savefig('General_histograms.pdf', dpi=300)

In [ ]:
'''
Normality test
'''
normality = pg.normality(X, method='normaltest')
#normality.to_csv('normality_test_features.csv')
normality